1.   請自行儲存復本修改
2.   按"執行階段"/"全部執行"
3.   或者每格順序執行
4.   ema,timeframe參數可自行修改
5.   抓取FTX全部USD交易對,再檢查在設定的timeframe是否出現黃金交叉,再畫出線圖.
6.   台股和美股要在colab中手動跑請確認scan_timelimit參數為False(不勾選). True(勾選)會限制每天只會在特定時間執行一次.
7.   可以自行修改放到Deepnote跑telegram bot.
8.    測試用的telegram channel: https://t.me/neoxbot_test 
9.    台股美股測試用telegram channel: https://t.me/vegas_tunnel_tw_us_stock_test


In [94]:
#@title User Setting { display-mode: "both" }

#ema setting
ema_s = 12 #@param {type:"integer"}
ema_m1 = 55 #@param {type:"integer"}
ema_m2 = 76 #@param {type:"integer"}
ema_h1 = 144 #@param {type:"integer"}
ema_h2 = 169 #@param {type:"integer"}
sma_s = 20 #@param {type:"integer"}

strategy = "KD_over" #@param ["ema_cross", "sma_cross","KD_over"]

kd_over_bought = 90
kd_over_sold = 10

scan_crypto = False #@param {type:"boolean"}
#ftx supprted timeframe ['15s', '1m', '5m', '15m', '1h', '4h', '1d', '3d', '1w', '2w', '1M']
t_frame = "1h" #@param ['1h', '4h', '1d', '3d']

scan_twstock = False #@param {type:"boolean"}
scan_usstock = True #@param {type:"boolean"}
#@markdown 在colab中執行時請確認scan_timelimlt為False
scan_timelimit = False #@param {type:"boolean"}

#線圖的佈景 https://plotly.com/python/templates/
plotly_theme = "plotly_dark" #@param ["plotly_dark", "plotly_white", "plotly", "ggplot2", "seaborn", "simple_white", "none"]

#子圖的設定
sub_plot_1 = "KD" #@param ["KD","MACD",""]
sub_plot_2 = "" #@param ["KD","MACD",""]

#@markdown Telegram bot setting
bot_token = '5381171826:AAH_LINwZ5gtECYXSlXA6mMZvKUMZe13RXc' #@param {type:"string"}
chat_id = '-1001759879229' #@param {type:"string"}
#@markdown Line Notify bot setting
line_notify_token  = "CclWwNgG6qbD5qx8eO3Oi4ii9azHfolj17SCzIE9UyI" #@param {type:"string"}

scan_market=[]
if scan_crypto:
    scan_market.append("scan_crypto")
if scan_twstock:
    scan_market.append("scan_twstock") 
if scan_usstock:
    scan_market.append("scan_usstock")        

Deepnote 無法安裝talib-binary, 需要特別的安裝方式.參考[用 Python 快速計算 158 種技術指標](https://medium.com/ai%E8%82%A1%E4%BB%94/%E7%94%A8-python-%E5%BF%AB%E9%80%9F%E8%A8%88%E7%AE%97-158-%E7%A8%AE%E6%8A%80%E8%A1%93%E6%8C%87%E6%A8%99-26f9579b8f3a)



In [95]:
#刪舊的png圖檔
import os
if os.name == "posix":
    !rm -f *.png

#安裝套件

!pip install jedi

try:
    import kaleido
except ModuleNotFoundError as e:
    !pip install -U kaleido    

try:
    import talib
except ModuleNotFoundError as e:
    if os.environ.get("COLAB_GPU",-1) != -1:
        !pip install talib_binary
    else:
        if os.path.exists("./talib"):
            %cd ta-lib
        else:    
            !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
            !tar -xzvf ta-lib-0.4.0-src.tar.gz
            %cd ta-lib
        !./configure --prefix=/usr
        !make
        !make install
        !pip install Ta-Lib
        %cd ..
        !rm -f ta-lib-0.4.0-src.tar.gz   
        !rm -rf ta-lib   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

#tw us stock from yfinance
try: 
    import yfinance as yf
except ModuleNotFoundError as e:
    !pip install yfinance
     
#request_cache for yfinance
try: 
    import requests_cache
except ModuleNotFoundError as e:
    !pip install requests_cache 

# 進行Plotly繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
if os.name == "posix" and not os.path.exists(path="/usr/share/fonts/truetype/TSbeta/TaipeiSansTCBeta-Regular.ttf"):
    !mkdir "/usr/share/fonts/truetype/TSbeta"
    !wget -O /usr/share/fonts/truetype/TSbeta/TaipeiSansTCBeta-Regular.ttf "https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download"

In [96]:
#載入模組

import talib
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time
import ccxt
import plotly.graph_objects as px
from plotly.subplots import make_subplots

import yfinance as yf   
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
#使用有cache的yfinance方式
#ticker = yf.Ticker('AAPL', session=session)
#ticker.history(period='1y',interval='1d')

- 台股交易時間為9:00至13:30，委託時間8:30至13:30

- 美股交易時間是台灣的晚上9點半至翌日凌晨4點收盤；冬令時間則是晚上10點半開盤至翌日凌晨5點收盤

底下為限制
1.台股只會在每週1,2,3,4,5 的18點可以被執行
2.美股只會在每周2,3,4,5,6 的7點被執行. 

如果調整在同一個時間點, 可能會因為yahoo finance的流量限制而卡住.

In [97]:
#收盤時間
#台股為日線資料, 每日跑一次即可

if scan_timelimit:

    tzone = dt.timezone(dt.timedelta(hours=8))
    now = dt.datetime.now(tz=tzone)

    #台股每週1,2,3,4,5, 每日的18點
    limit_hour = (18,)
    limit_weekday = (1,2,3,4,5)
    if now.hour not in limit_hour or now.isoweekday() not in limit_weekday:
        if "scan_twstock" in scan_market:
            scan_market.remove("scan_twstock")

    #美股每週2,3,4,5,6 每日的7點
    limit_hour = (7,)
    limit_weekday = (2,3,4,5,6)
    if now.hour not in limit_hour or now.isoweekday() not in limit_weekday:
        if "scan_usstock" in scan_market:
            scan_market.remove("scan_usstock")

In [98]:
#crypto
#獲取FTX的現貨/USD交易對

if 'scan_crypto' in scan_market:

    exchange = ccxt.ftx()
    exchange.load_markets()
    symbol_list = set()
    ex_symbol_list = set()
    #排除底下字串
    ex_string=("USD:USD","BULL","BEAR","HALF","HEDGE","JPY","EUR","TRYB","AUD","/BTC","/DOGE","/USDT","GBP","HKD","ZAR","CAD","BRZ","MXN","ARS","DAI")
    for i in exchange.symbols:
        for j in ex_string:
            if i.find(j) != -1:
                ex_symbol_list.add(i)

    crypto_symbol_list = set(exchange.symbols) - ex_symbol_list #set差集運算

    #print(len(symbol_list))
    #print(symbol_list)
        

In [99]:
#crypto
#自定義交易對列表 2022/08/26
#如果要使用自定的交易對,請把symbol_inFTX 改成 symbol_list.
if "scan_crypto" in scan_market:
        crypto_symbol_inFTX = ['1INCH/USD', 'AAPL/USD', 'AAVE/USD', 'ABNB/USD', 'ACB/USD', 'AGLD/USD', 'AKRO/USD', 'ALCX/USD',
        'ALEPH/USD', 'ALGO/USD', 'ALICE/USD', 'ALPHA/USD', 'AMC Entertainment Holdings/USD',
        'AMD/USD', 'AMPL/USD', 'AMZN/USD', 'ANC/USD', 'APE/USD', 'APHA/USD', 'ARKK/USD',
        'ASD/USD', 'ATLAS/USD', 'ATOM/USD', 'AURY/USD', 'AVAX/USD', 'AXS/USD', 'BABA/USD',
        'BADGER/USD', 'BAL/USD', 'BAND/USD', 'BAO/USD', 'BAR/USD', 'BAT/USD', 'BB/USD',
        'BCH/USD', 'BICO/USD', 'BILI/USD', 'BIT/USD', 'BITO/USD', 'BITW/USD', 'BLT/USD',
        'BNB/USD', 'BNT/USD', 'BNTX/USD', 'BOBA/USD', 'BTC/USD', 'BTT/USD', 'BVOL/USD',
        'BYND/USD', 'C98/USD', 'CEL/USD', 'CGC/USD', 'CHR/USD', 'CHZ/USD', 'CITY/USD',
        'CLV/USD', 'COIN/USD', 'COMP/USD', 'CONV/USD', 'COPE/USD', 'CQT/USD', 'CREAM/USD',
        'CRO/USD', 'CRON/USD', 'CRV/USD', 'CTX/USD', 'CUSDT/USD', 'CVC/USD', 'CVX/USD',
        'DAWN/USD', 'DENT/USD', 'DFL/USD', 'DKNG/USD', 'DMG/USD', 'DODO/USD', 'DOGE/USD',
        'DOT/USD', 'DYDX/USD', 'EDEN/USD', 'EMB/USD', 'ENJ/USD', 'ENS/USD', 'ETH/USD',
        'ETHE/USD', 'FB/USD', 'FIDA/USD', 'FRONT/USD', 'FTM/USD', 'FTT/USD', 'FXS/USD',
        'GAL/USD', 'GALA/USD', 'GALFAN/USD', 'GARI/USD', 'GBTC/USD', 'GDX/USD', 'GDXJ/USD',
        'GENE/USD', 'GLD/USD', 'GLXY/USD', 'GME/USD', 'GMT/USD', 'GODS/USD', 'GOG/USD',
        'GOOGL/USD', 'GRT/USD', 'GST/USD', 'GT/USD', 'HGET/USD', 'HMT/USD', 'HNT/USD',
        'HOLY/USD', 'HOOD/USD', 'HT/USD', 'HUM/USD', 'HXRO/USD', 'IBVOL/USD', 'IMX/USD',
        'INDI/USD', 'INTER/USD', 'IP3/USD', 'JET/USD', 'JOE/USD', 'JST/USD', 'KBTT/USD',
        'KIN/USD', 'KNC/USD', 'KSHIB/USD', 'KSOS/USD', 'LDO/USD', 'LEO/USD', 'LINA/USD',
        'LINK/USD', 'LOOKS/USD', 'LRC/USD', 'LTC/USD', 'LUA/USD', 'MANA/USD', 'MAPS/USD',
        'MATH/USD', 'MATIC/USD', 'MBS/USD', 'MCB/USD', 'MEDIA/USD', 'MER/USD', 'MKR/USD',
        'MNGO/USD', 'MOB/USD', 'MRNA/USD', 'MSOL/USD', 'MSTR/USD', 'MTA/USD', 'MTL/USD',
        'NEAR/USD', 'NEXO/USD', 'NFLX/USD', 'NIO/USD', 'NOK/USD', 'NVDA/USD', 'OKB/USD',
        'OMG/USD', 'ORBS/USD', 'ORCA/USD', 'OXY/USD', 'PAXG/USD', 'PENN/USD', 'PEOPLE/USD',
        'PERP/USD', 'PFE/USD', 'POLIS/USD', 'PORT/USD', 'PRISM/USD', 'PROM/USD', 'PSG/USD',
        'PSY/USD', 'PTU/USD', 'PUNDIX/USD', 'PYPL/USD', 'QI/USD', 'RAY/USD', 'REAL/USD',
        'REEF/USD', 'REN/USD', 'RNDR/USD', 'ROOK/USD', 'RSR/USD', 'SAND/USD', 'SECO/USD',
        'SHIB/USD', 'SKL/USD', 'SLND/USD', 'SLP/USD', 'SLRS/USD', 'SLV/USD', 'SNX/USD',
        'SNY/USD', 'SOL/USD', 'SOS/USD', 'SPA/USD', 'SPELL/USD', 'SPY/USD', 'SQ/USD',
        'SRM/USD', 'STEP/USD', 'STETH/USD', 'STG/USD', 'STMX/USD', 'STORJ/USD', 'STSOL/USD',
        'SUN/USD', 'SUSHI/USD', 'SXP/USD', 'StarLaunch/USD', 'TLM/USD', 'TLRY/USD', 'TOMO/USD',
        'TONCOIN/USD', 'TRU/USD', 'TRX/USD', 'TSLA/USD', 'TSM/USD', 'TULIP/USD', 'TWTR/USD',
        'UBER/USD', 'UBXT/USD', 'UMEE/USD', 'UNI/USD', 'USDT/USD', 'USO/USD', 'VGX/USD',
        'WAVES/USD', 'WBTC/USD', 'WtFLOW/USD', 'WNDR/USD', 'WRX/USD', 'XAUT/USD', 'XRP/USD',
        'YFI/USD', 'YFII/USD', 'YGG/USD', 'ZM/USD', 'ZRX/USD']
 

1.  台股代號列表抓證交所的.
2.  台股資料抓yahoo finance的, 證交所的太慢又限制流量太嚴重.
3.  美股代號只抓S&P500的成分股.
4.  美股資料通用yahoo finance.

[台股美股用yfinance(yahoo finance)套件](https://www.finlab.tw/%E7%94%A8%E7%88%AC%E8%9F%B2%E7%88%AC%E5%85%A8%E4%B8%96%E7%95%8C%E8%82%A1%E5%83%B9/)

In [100]:
#取得股票代號
if "scan_twstock" in scan_market:
    twstock_symbol_list=[]
    twse_mode = (2,4,5)  #2=上市, 4=上櫃, 5=興櫃

    for i in twse_mode:
        res = requests.get(f"http://isin.twse.com.tw/isin/C_public.jsp?strMode={i}")
        df = pd.read_html(res.text)[0]
        df.columns = ['symbol','ISINCode','start','market','type','CFICode','remakr']
        df = df.iloc[2:]
        df = df.reset_index()
        for j in range(df.shape[0]):
            if df.at[j,'CFICode'] == 'ESVUFR':
                if i in (2,):
                    twstock_symbol_list.append(df.at[j,'symbol'].replace('\u3000','.tw/'))
                elif i in (4,5):
                    twstock_symbol_list.append(df.at[j,'symbol'].replace('\u3000','.two/'))
    print(f"台股數量(上市,上櫃,興櫃):{len(twstock_symbol_list)}")

In [101]:
#twstock
#自定義交易對列表 
#如果要使用自定的交易對,請把symbol_mylist 改成 symbol_list.
#使用yahoo finance來抓資料, 上市代號後要加.tw， 上櫃興櫃代號要加.two
if "scan_twstock" in scan_market:
    twstock_symbol_mylist = ['2357.tw/華碩', '2317.tw/精英', '6797.two/圓點']

In [102]:
#usstock S&P500 list
if "scan_usstock" in scan_market:
    res = requests.get(f"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    df = pd.read_html(res.text)[0]

    usstock_symbol_list = [f"{df.at[i,'Symbol']}/{df.at[i,'Security']}" for i in range(df.shape[0])]

#print(usstock_symbol_list)

In [103]:
#us stock
#美股自定義交易對列表
#如果要使用自定的交易對,請把symbol_mylist 改成 symbol_list.
if "scan_usstock" in scan_market:
    usstock_symbol_mylist = ['TSLA/Tesla', 'AAPL/Apple']

### plotly畫K線,KD,MACD

[A simple guide to plotly](https://python.plainenglish.io/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)
### telegram bot sendphoto
[telegram api sendphoto](https://core.telegram.org/bots/api#sendphoto)

[code sample](https://stackoverflow.com/questions/36778321/how-to-send-photo-on-telegram-bot)

### Line notify bot
[Line notify api doc](https://notify-bot.line.me/doc/en/)

In [104]:
#define function
#
#get ohlc_data
def get_crypto_ohlc_data(symbol, timeframe="1h"):
    try:
        ohlc_data = exchange.fetch_ohlcv(symbol,timeframe=timeframe,limit=400)
    except Exception as e:
        print(symbol + ":" + e)

    for i in ohlc_data:
        i[0] = dt.datetime.fromtimestamp(i[0]/1000)

    ohlc_df = pd.DataFrame(ohlc_data,columns=["date","open","high","low","close","volume"]) 
    
    ohlc_df.date = pd.to_datetime(ohlc_df.date)
    
    ema_sline = talib.EMA(ohlc_df["close"],timeperiod=ema_s)
    ema_m1line = talib.EMA(ohlc_df["close"],timeperiod=ema_m1)
    ema_m2line = talib.EMA(ohlc_df["close"],timeperiod=ema_m2)
    ema_h1line = talib.EMA(ohlc_df["close"],timeperiod=ema_h1)
    ema_h2line = talib.EMA(ohlc_df["close"],timeperiod=ema_h2)
    sma_sline = talib.SMA(ohlc_df["close"],timeperiod=sma_s)
    #KD
    slowk,slowd = talib.STOCH(high=ohlc_df["high"],low=ohlc_df["low"],close=ohlc_df["close"],fastk_period=9, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    #MACD
    macd, macdsignal, macdhist = talib.MACD(ohlc_df["close"], fastperiod=12, slowperiod=26, signalperiod=9)

    ohlc_df['ema_sline'] = ema_sline
    ohlc_df['ema_m1line'] = ema_m1line
    ohlc_df['ema_m2line'] = ema_m2line
    ohlc_df['ema_h1line'] = ema_h1line
    ohlc_df['ema_h2line'] = ema_h2line
    ohlc_df['sma_sline'] = sma_sline
    ohlc_df['slowk'] = slowk
    ohlc_df['slowd'] = slowd
    ohlc_df['macd'] = macd
    ohlc_df['macdsignal'] = macdsignal
    ohlc_df['macdhist'] = macdhist

    data = {
        'symbol' : symbol,
        'ohlc_df' : ohlc_df.tail(150),
        'timeframe' : timeframe,
        'market' : "crypto"
    }

    time.sleep(100/1000)
    return data

def get_yfstock_ohlc_data(symbol):
    error_msg=""
    symbol_code = symbol.split("/")[0]

    #使用request_cache會造成讀取到舊資料.
    #只能在本機和colab上使用request_cache
    if os.environ.get("COLAB_GPU",-1) != -1 or os.environ.get("COMPUTERNAME") == "ZENBOOK-NEO":
        ticker = yf.Ticker(ticker=symbol_code,session=session)
    else:    
        ticker = yf.Ticker(ticker=symbol_code)
        
    ohlc_df = ticker.history(tickers=symbol_code,period='1y',intervan='1d')
    
    ohlc_df = ohlc_df.reset_index()
    ohlc_df.columns = ['date','open','high','low','close','volume','dividends','stocksplits']

    ema_sline = talib.EMA(ohlc_df["close"],timeperiod=ema_s)
    ema_m1line = talib.EMA(ohlc_df["close"],timeperiod=ema_m1)
    ema_m2line = talib.EMA(ohlc_df["close"],timeperiod=ema_m2)
    ema_h1line = talib.EMA(ohlc_df["close"],timeperiod=ema_h1)
    ema_h2line = talib.EMA(ohlc_df["close"],timeperiod=ema_h2)    
    sma_sline = talib.SMA(ohlc_df["close"],timeperiod=sma_s)
    #KD
    slowk,slowd = talib.STOCH(high=ohlc_df["high"],low=ohlc_df["low"],close=ohlc_df["close"],fastk_period=9, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    #MACD
    macd, macdsignal, macdhist = talib.MACD(ohlc_df["close"], fastperiod=12, slowperiod=26, signalperiod=9)


    ohlc_df['ema_sline'] = ema_sline
    ohlc_df['ema_m1line'] = ema_m1line
    ohlc_df['ema_m2line'] = ema_m2line
    ohlc_df['ema_h1line'] = ema_h1line
    ohlc_df['ema_h2line'] = ema_h2line
    ohlc_df['sma_sline'] = sma_sline
    ohlc_df['slowk'] = slowk
    ohlc_df['slowd'] = slowd
    ohlc_df['macd'] = macd
    ohlc_df['macdsignal'] = macdsignal
    ohlc_df['macdhist'] = macdhist

    data = {
        'symbol' : symbol,
        'ohlc_df' : ohlc_df.tail(150),
        'timeframe' : "1d",
        'market' : "twstock"
    }

    time.sleep(100/1000)
    return data


#檢查黃金交差, ema_s向上交叉ema_m1或ema_m2,選最高者
def check_bullorbear_ema(data):
    bull_or_bear = ""
    if data["ohlc_df"]["ema_m1line"].iat[-1] > data["ohlc_df"]["ema_m2line"].iat[-1]:
        if data["ohlc_df"]["ema_sline"].iat[-1] > data["ohlc_df"]["ema_m1line"].iat[-1] \
            and data["ohlc_df"]["ema_sline"].iat[-2] < data["ohlc_df"]["ema_m1line"].iat[-2]:
            bull_or_bear = "bull"
        elif  data["ohlc_df"]["ema_sline"].iat[-1] < data["ohlc_df"]["ema_m2line"].iat[-1] \
            and data["ohlc_df"]["ema_sline"].iat[-2] > data["ohlc_df"]["ema_m2line"].iat[-2]:
            bull_or_bear = "bear"    
    else:
        if data["ohlc_df"]["ema_sline"].iat[-1] > data["ohlc_df"]["ema_m2line"].iat[-1] \
            and data["ohlc_df"]["ema_sline"].iat[-2] < data["ohlc_df"]["ema_m2line"].iat[-2]:
            bull_or_bear = "bull"
        elif data["ohlc_df"]["ema_sline"].iat[-1] < data["ohlc_df"]["ema_m1line"].iat[-1] \
            and data["ohlc_df"]["ema_sline"].iat[-2] > data["ohlc_df"]["ema_m1line"].iat[-2]: 
            bull_or_bear = "bear" 

    return bull_or_bear

#檢查黃金交差, ema_s向上交叉sma_s
def check_bullorbear_sma(data):
    bull_or_bear = ""
    if data["ohlc_df"]["sma_sline"].iat[-1] > data["ohlc_df"]["ema_sline"].iat[-1] \
        and data["ohlc_df"]["sma_sline"].iat[-2] < data["ohlc_df"]["ema_sline"].iat[-2]:
        bull_or_bear = "bull"
    elif data["ohlc_df"]["sma_sline"].iat[-1] < data["ohlc_df"]["ema_sline"].iat[-1] \
        and data["ohlc_df"]["sma_sline"].iat[-2] > data["ohlc_df"]["ema_sline"].iat[-2]:
        bull_or_bear = "bear"    

    return bull_or_bear


#KD_over_bought_sold
#KD超買超賣
def check_bullorbear_kd(data):
    bull_or_bear = ""
    if data["ohlc_df"]["slowk"].iat[-1] > kd_over_bought and data["ohlc_df"]["slowd"].iat[-1] > kd_over_bought :
        bull_or_bear = "bear"
    elif data["ohlc_df"]["slowk"].iat[-1] < kd_over_sold and  data["ohlc_df"]["slowd"].iat[-1] < kd_over_sold:
        bull_or_bear = "bull"    
        
    return bull_or_bear

#畫K線
def plotly_kline(data):
    candlestick = px.Candlestick(x=data["ohlc_df"]["date"],
                    open=data["ohlc_df"]['open'],
                    high=data["ohlc_df"]['high'],
                    low=data["ohlc_df"]['low'],
                    close=data["ohlc_df"]['close'],name="Kline")

    ema_sline = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_sline'],
                    line=dict(color='orange', width=1),
                    mode='lines',name=f"EMA{ema_s}")
    
    ema_m1line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_m1line'],
                    line=dict(color='green', width=1),
                    mode='lines',name=f"EMA{ema_m1}")

    ema_m2line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_m2line'],
                    line=dict(color='blue', width=1),
                    mode='lines',name=f"EMA{ema_m2}")

    ema_h1line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_h1line'],
                    line=dict(color='pink', width=1),
                    mode='lines',name=f"EMA{ema_h1}")

    ema_h2line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_h2line'],
                    line=dict(color='red', width=1),
                    mode='lines',name=f"EMA{ema_h2}")

    sma_sline = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['sma_sline'],
                    line=dict(color='yellow', width=1),
                    mode='lines',name=f"SMA{sma_s}")
    
    sma_deduct_df=data['ohlc_df'][(0-sma_s):(0-sma_s+1)]
    #sma_deduct_df=data['ohlc_df'].iloc[0-sma_s]
    sma_deduct = px.Scatter(x=sma_deduct_df['date'],y=sma_deduct_df['close'],
                    mode='markers',name=f"SMA{sma_s}_deduct",
                    marker=(dict(symbol="arrow-bar-right",size=20)))

    #KD
    slowk = px.Scatter(x=data["ohlc_df"]["date"],
                   y=data["ohlc_df"]["slowk"], line=dict(color="yellow", width=1),
                    mode='lines',showlegend=False, name=f"slowK")                

    slowd = px.Scatter(x=data["ohlc_df"]["date"],
                   y=data["ohlc_df"]['slowd'], line=dict(color='green', width=1),
                    mode='lines',showlegend=False, name=f"slowd")                  
    stoch_up = px.Scatter(x=data["ohlc_df"]["date"],
                   y=[kd_over_bought for i in range(data["ohlc_df"]["date"].shape[0])] , line=dict(color='blue', width=0),
                    mode='lines', fill='tonexty',showlegend=False ,fillcolor='rgba(0,119,255,0.1)')                
    stoch_down = px.Scatter(x=data["ohlc_df"]["date"],
                   y=[kd_over_sold for i in range(data["ohlc_df"]["date"].shape[0])] ,showlegend=False ,line=dict(color='blue', width=0),
                    mode='lines') 
    stoch_up_arrow = px.Scatter(x=data["ohlc_df"]["date"],
                    y=[100 if val > kd_over_bought else None for val in data["ohlc_df"]["slowd"]],
                    mode="markers",name=f"kd over{kd_over_bought}", showlegend=True, marker=(dict(symbol="arrow-down",size=10,color='red'))) 
    stoch_down_arrow = px.Scatter(x=data["ohlc_df"]["date"],
                    y=[0 if val < kd_over_sold else None for val in data["ohlc_df"]["slowd"]],
                    mode="markers",name=f"kd over{kd_over_sold}", showlegend=True, marker=(dict(symbol="arrow-up",size=10,color='green')))                                      
    #macd
    macd = px.Scatter(x=data['ohlc_df']['date'],
                    y=data['ohlc_df']['macd'],line=dict(color='red', width=1),
                    mode='lines',showlegend=False)
    macdsignal = px.Scatter(x=data['ohlc_df']['date'],
                    y=data['ohlc_df']['macdsignal'],line=dict(color="blue", width=1),
                    mode='lines',showlegend=False)
    colors = [ "green" if bar > 0  else 'red' for bar in data['ohlc_df']['macdhist'] ]               
    macdhist = px.Bar(x=data['ohlc_df']['date'],
                y=data['ohlc_df']['macdhist'],marker_color=colors,showlegend=False)                                  
                    

    title_text = f"{data['market']} {data['result']} 時框:{data['timeframe']} Last update: {data['ohlc_df']['date'].iat[-1]}"
    
    if sub_plot_1 != "" and sub_plot_2 != "":
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,row_heights=[0.6,0.2,0.2],vertical_spacing=0.01)
        fig.add_traces([sma_sline,ema_sline,ema_m1line,ema_m2line,ema_h1line,ema_h2line,candlestick,sma_deduct],rows=1,cols=1)
        
        if sub_plot_1 == "KD":
            fig.add_traces([stoch_down,stoch_up,slowk,slowd,stoch_down_arrow,stoch_up_arrow],rows=2,cols=1)
            fig.update_yaxes(title_text="KD",row=2,col=1)
        elif sub_plot_1 == "MACD":
            fig.add_traces([macd,macdsignal,macdhist],rows=2,cols=1)
            fig.update_yaxes(title_text="MACD",row=2,col=1)

        if sub_plot_2 == "KD":
            fig.add_traces([stoch_down,stoch_up,slowk,slowd,stoch_down_arrow,stoch_up_arrow],rows=3,cols=1)
            fig.update_yaxes(title_text="KD",row=3,col=1)
        elif sub_plot_2 == "MACD":
            fig.add_traces([macd,macdsignal,macdhist],rows=3,cols=1)
            fig.update_yaxes(title_text="MACD",row=3,col=1)     

    elif sub_plot_1 != "" or sub_plot_2 != "":
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[0.7,0.3],vertical_spacing=0.01)
        fig.add_traces([sma_sline,ema_sline,ema_m1line,ema_m2line,ema_h1line,ema_h2line,candlestick,sma_deduct],rows=1,cols=1)
        if sub_plot_1 == "KD":
            fig.add_traces([stoch_down,stoch_up,slowk,slowd,stoch_down_arrow,stoch_up_arrow],rows=2,cols=1)
            fig.update_yaxes(title_text="KD",row=2,col=1)
        elif sub_plot_1 == "MACD":
            fig.add_traces([macd,macdsignal,macdhist],rows=2,cols=1)
            fig.update_yaxes(title_text="MACD",row=2,col=1)
        elif sub_plot_2 == "KD":
            fig.add_traces([stoch_down,stoch_up,slowk,slowd,stoch_down_arrow,stoch_up_arrow],rows=2,cols=1)
            fig.update_yaxes(title_text="KD",row=2,col=1)
        elif sub_plot_2 == "MACD":
            fig.add_traces([macd,macdsignal,macdhist],rows=2,cols=1)
            fig.update_yaxes(title_text="MACD",row=2,col=1)     

    elif sub_plot_1 == "" or sub_plot_2 == "":
        fig = make_subplots(rows=1, cols=1, shared_xaxes=True,row_heights=[1])
        fig.add_traces([sma_sline,ema_sline,ema_m1line,ema_m2line,ema_h1line,ema_h2line,candlestick,sma_deduct],rows=1,cols=1)

    
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[0.7,0.3])
    #make_subplots為建立子圖
    #fig = px.Figure(data=[candlestick,ema_sline,ema_m1line])
    
    fig.add_traces([sma_sline,ema_sline,ema_m1line,ema_m2line,ema_h1line,ema_h2line,candlestick,sma_deduct],rows=1,cols=1)
    fig.add_traces([stoch_down,stoch_up,slowk,slowd,stoch_down_arrow,stoch_up_arrow],rows=2,cols=1)
    #
    #fig.add_traces([macd,macdsignal,macdhist],rows=2,cols=1)

    """
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(title=title_text,title_font_family="Taipei Sans TC Beta",autosize=False,width=1000,height=450)
    #fig.update_yaxes(title_text="KD",row=2,col=1)
    fig["layout"]["template"]=plotly_theme

    fig.show()
    fig.write_image(f"{data['symbol'].split('/')[0]}.png")

def send_to_bot_msg(tid_, msg, bot_token_):
    url = f'https://api.telegram.org/bot{bot_token_}/sendMessage'
    headers = {'content-type': 'application/json'}
    data  = f'{{"chat_id": "{tid_}", "text": "{msg}", "parse_mode": "HTML", "disable_web_page_preview": "true"}}'.encode('utf-8')
    r = requests.post(url, data=data, headers=headers, verify=False)  

def send_to_bot_photo(tid_, msg, bot_token_, photo_opened):
    url = f'https://api.telegram.org/bot{bot_token_}/sendPhoto'
    headers = {'content-type':'multipart/form-data'}
    data  = {"chat_id": tid_, "caption": msg, "parse_mode": "HTML", "disable_web_page_preview": "true"}
    photo_data = {'photo': photo_opened}
    r = requests.post(url, data=data, files=photo_data) 

def send_to_line_notify_bot(msg, line_notify_token, photo_opened):
    line_notify_token = "CclWwNgG6qbD5qx8eO3Oi4ii9azHfolj17SCzIE9UyI"
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization": f"Bearer {line_notify_token}"}
    data = {"message":msg}
    image_file = {'imageFile': photo_opened}
    r = requests.post(url=url,data=data,headers=headers,files=image_file)
    
  

### 解決Plotly 在linux中畫圖存檔無法正確顥示中文問題.
1.下載中文字型檔.
2.Plotly裡設定使用字型.

[下載中文字型](https://colab.research.google.com/github/willismax/matplotlib_show_chinese_in_colab/blob/master/matplotlib_show_chinese_in_colab.ipynb#scrollTo=vw9ce7WO6ePN)

[Plotly字型設定](https://plotly.com/python/figure-labels/)

### 如何建立telegram bot頻道
1. 找BotFather建立機器人,取得token. 請參考[Day 16-開始製作Telegram聊天機器人！](https://ithelp.ithome.com.tw/articles/10244411)

2. 建立一個頻道,並把機器人加進去,取得chat_id.每一個對話, 群組, 頻道都是一個chat_id, 請參考[取得chat_id](http://blog.3dgowl.com/telegram-telegram%E4%BA%94-%E5%8F%96%E5%BE%97-chat-id/)

3. 在開頭填入token 和chat_id.

4. 下載這份程式為*.ipynb檔案.

5. 到deepnote.com建立帳號,新增一個project ,把*,ipynb上傳. 

6. 按Run notebook可以手動執行, 按右方小鍵頭可以排程.timeframe 1h就每小時跑，1h以上就依你的時框去排程, 如果timeframe 1d 排每小時跑, 應該會一直收到重覆訊息.

### Line notify bot
[Line notify api doc](https://notify-bot.line.me/doc/en/)


In [105]:
#start 

for i in scan_market:
    ta_data = {}
    
    if i == 'scan_crypto':
        symbol_list = crypto_symbol_list
    elif i == 'scan_twstock':
        symbol_list = twstock_symbol_list
    elif i == 'scan_usstock':
        symbol_list = usstock_symbol_list
        

    for j in symbol_list:
        try:
            if i == 'scan_crypto':
                ta_data = get_crypto_ohlc_data(j)
                ta_data['market'] = "Crypto"
                ta_data['emoji'] = "₿"
            elif i == 'scan_twstock':
                ta_data = get_yfstock_ohlc_data(j)
                ta_data['market'] = "TW stock"
                ta_data['emoji'] = "🇹🇼"
            elif i == "scan_usstock":
                ta_data = get_yfstock_ohlc_data(j)
                ta_data['market'] = "US stock" 
                ta_data['emoji'] = "🇺🇸"

            if strategy == 'ema_cross':  
                if check_bullorbear_ema(ta_data) == 'bull':
                    ta_data["emoji"] += "💰💰"
                    ta_data["result"] = f"#{j.replace('/', ' /')} ema is Golden Cross!!" 
                elif check_bullorbear_ema(ta_data) == 'bear':
                    ta_data["emoji"] += "☠️☠️"
                    ta_data["result"] = f"#{j.replace('/', ' /')} ema is Dead Cross!!"
                else:
                    ta_data["result"] = ""

            elif strategy == 'sma_cross':
                if check_bullorbear_sma(ta_data) == 'bull':
                    ta_data["emoji"] += "💰💰"
                    ta_data["result"] = f"#{j.replace('/', ' /')} sma is Golden Cross!!" 
                elif check_bullorbear_ema(ta_data) == 'bear':
                    ta_data["emoji"] += "☠️☠️"
                    ta_data["result"] = f"#{j.replace('/', ' /')} sma is Dead Cross!!"
                else:
                    ta_data["result"] = ""

            elif  strategy == 'KD_over': 
                if check_bullorbear_kd(ta_data) == 'bull':
                    ta_data["emoji"] += "💰💰"
                    ta_data["result"] = f"#{j.replace('/', ' /')} kd is over sold!!" 
                elif check_bullorbear_kd(ta_data) == 'bear':
                    ta_data["emoji"] += "☠️☠️"
                    ta_data["result"] = f"#{j.replace('/', ' /')} kd is over bought!!"
                else:
                    ta_data["result"] = ""

            if ta_data['result']:
                print(f"{ta_data['emoji']} {ta_data['symbol']} {ta_data['result']}")   
                plotly_kline(ta_data)    
                #send photo
                if bot_token:
                    send_to_bot_photo(tid_=chat_id, msg=f"{ta_data['emoji']} {ta_data['result']}", bot_token_=bot_token, photo_opened=open(f"{j.split('/')[0]}.png", "rb"))
                if line_notify_token:
                    send_to_line_notify_bot(msg=f"{ta_data['emoji']} {ta_data['result']}",line_notify_token=line_notify_token,photo_opened=open(f"{j.split('/')[0]}.png", "rb"))
        except Exception as e:
            print(e)    

🇺🇸💰💰 MMM/3M #MMM /3M kd is over sold!!


🇺🇸💰💰 AOS/A. O. Smith #AOS /A. O. Smith kd is over sold!!


🇺🇸💰💰 ACN/Accenture #ACN /Accenture kd is over sold!!


KeyboardInterrupt: 